In [8]:
#need gensim 3.8.3
#
#


In [9]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# 去除停用词
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# 去除只出现一次的词
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
#处理后的Bag-of-words文本
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

print(texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [10]:
#建立字典
dictionary = corpora.Dictionary(texts)
print(dictionary.token2id)
print()
#基于上述字典建立corpus
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [11]:
from gensim import models
#tf-idf表达

#初始化tf-idf模型，主要是计算IDF
tfidf = models.TfidfModel(corpus)  
print(tfidf)

TfidfModel(num_docs=9, num_nnz=28)


In [12]:
#得到每一个文档的TF-IDF表达，以稀疏矩阵的形式存储
corpus_tfidf = tfidf[corpus]
for doc, as_text in zip(corpus_tfidf, documents):
    print(doc, as_text)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)] Human machine interface for lab abc computer applications
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)] A survey of user opinion of computer system response time
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)] The EPS user interface management system
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)] System and human system engineering testing of EPS
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)] Relation of user perceived response time to error measurement
[(9, 1.0)] The generation of random binary unordered trees
[(9, 0.7071067811865475), (10, 0.7071067811865475)] The intersection graph of paths in trees
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)] Graph minors I

In [13]:
#运行LSI, 以TFIDF表达为输入，2个话题
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # 初始化LSI模型参数, K=2
corpus_lsi = lsi_model[corpus_tfidf]  #基于corpus_tfidf训练LSI模型

#打印出学习到的latent topic，一共两个
lsi_model.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [14]:
# 打印文档的topic表达，每一个文档表示成2维的topic向量
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090366), (1, -0.5200703306361854)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142444), (1, -0.7609563167700055)] A survey of user opinion of computer system response time
[(0, 0.08992639972446359), (1, -0.7241860626752513)] The EPS user interface management system
[(0, 0.07585847652178086), (1, -0.6320551586003433)] System and human system engineering testing of EPS
[(0, 0.10150299184980074), (1, -0.5737308483002961)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378316), (1, 0.16115180214025734)] The generation of random binary unordered trees
[(0, 0.8774787673119837), (1, 0.1675890686465934)] The intersection graph of paths in trees
[(0, 0.9098624686818584), (1, 0.14086553628718942)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569283), (1, -0.053929075663894294)] Graph minors A survey


In [15]:

from gensim.models import nmf
corpus_nmf = nmf.Nmf(corpus_tfidf, num_topics=2)
print(corpus_nmf)
corpus_nmf.print_topics(2)

[(0,
  '0.139*"10" + 0.135*"4" + 0.119*"6" + 0.119*"3" + 0.116*"11" + 0.112*"7" + 0.093*"9" + 0.089*"5" + 0.050*"0" + 0.029*"8"'),
 (1,
  '0.247*"2" + 0.238*"1" + 0.195*"8" + 0.178*"5" + 0.109*"0" + 0.032*"7" + 0.000*"10" + 0.000*"3" + 0.000*"4" + 0.000*"9"')]

In [16]:
print(corpus_nmf.get_term_topics(word_id=0))#得到某一个词的topic表达，输入为词id

print(corpus_nmf.get_document_topics([(0, 1), (1, 1), (2, 1)])) #得到文档的topic表示，输入为文档的bag of words表达

[(0, 0.3802730849132547), (1, 0.6197269150867454)]
[(1, 1.0)]


In [17]:
# 基于texts数据集合训练一个word2vec模型, 隐维度为2
import gensim
print(texts)
w2v = gensim.models.Word2Vec(texts, min_count=1, size= 2)
print(w2v)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
Word2Vec(vocab=12, size=2, alpha=0.025)


C:\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  elif isinstance(sentences, GeneratorType):


In [18]:
#查询词向量
for i, word in enumerate(w2v.wv.vocab):
    if i == 20:
        break
    print(word, ":", w2v[word])

human : [-0.13661432 -0.18003137]
interface : [-0.0580059   0.15071844]
computer : [-0.02643474  0.08309188]
survey : [ 0.17961481 -0.02941382]
user : [-0.1339317  -0.17625459]
system : [-0.19018722 -0.22744949]
response : [ 0.15072194 -0.20308909]
time : [0.1087472  0.23340051]
eps : [ 0.20113502 -0.0439257 ]
trees : [0.19270732 0.20145503]
graph : [ 0.1950712  -0.01611915]
minors : [-0.04270541 -0.23165946]


<ipython-input-18-89f710c2158c>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  print(word, ":", w2v[word])


In [19]:
# 计算两个词的相似度
pairs = [
    ('human', 'computer'),   
    ('interface', 'computer'),  
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, w2v.wv.similarity(w1, w2)))

'human'	'computer'	-0.58
'interface'	'computer'	1.00
